# Introduction
WIth the dataset including over 185,000 samples we greatly reduced the size of the training and testing samples while still maintaining a 1:1:1 ratio

In [4]:
!pip install tensorflow
!pip install tensorflow-hub

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


# Loading the Feature extraction VGG16 model

In [5]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model

#variable placeholders for image dimesnions
img_height = 256
img_width = 256

#drop top
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

#freeze layers
for layer in base_model.layers:
    layer.trainable = False

#feature extraction at last layer
output = base_model.output

#feature extraction model
feature_extraction_model = Model(inputs=base_model.input, outputs=output)

#model summary
print(feature_extraction_model.summary())


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 256, 256, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 256, 256, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 128, 128, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 128, 128, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 64, 64, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 64, 64, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 64, 64, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 32, 32, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 32, 32, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 32, 32, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 32, 32, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 16, 16, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 8, 8, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)

None


# Feature Extraction
Decision to extract features style by style as they are stored in folders

Create a function to test if the image is within the specified 256 x 256 pixels to normalize all inputs

In [6]:
from PIL import Image
import numpy as np

def preprocess_image(img_path):
    #target size of images256 x 256
    target_size = (256,256)

    img = Image.open(img_path)

    if img.size != target_size:
        img = img.resize(target_size, resample=Image.BICUBIC)

    #return the image array
    return (np.array(img))

Create a function to process all the images in a folder

In [9]:
import os
import time
import pandas as pd
from tensorflow.keras.applications.vgg16 import preprocess_input

def extract_features_from_folder(input_folder_path):
    count = 0
    csv_file_path = os.path.join(input_folder_path + '.csv')

    with open(csv_file_path, 'a') as f:
        for filename in os.listdir(input_folder_path):
            img_path = os.path.join(input_folder_path, filename)
            if os.path.isfile(img_path):
                #start_time = time.time()

                img_array = preprocess_image(img_path)
                img_array = np.expand_dims(img_array, axis=0)
                img_array = preprocess_input(img_array)

                features = feature_extraction_model.predict(img_array)

                pd.DataFrame(features.flatten()).transpose().to_csv(f, header=False, index=False)

                #end_time = time.time()
                print(count)
                count = count + 1
    
    f.close()

testing on a sample folder, remember to stop

In [10]:
extract_features_from_folder('train/AI_LD_art_nouveau')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 380ms/step
0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 366ms/step
1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 389ms/step
2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 357ms/step
3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 353ms/step
4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 360ms/step
5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 378ms/step
6
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 420ms/step
7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 422ms/step
8
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 364ms/step
9
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 368ms/step
10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 358ms/step
11
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 368ms/step
12
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 386ms/step
13
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 384ms/step
14
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 403ms/step
15
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 388ms/step
16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 364ms/step
17
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 386ms/step
18
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 376ms/step
19
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 391ms/step
20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 407ms/step
21
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 410ms/step
22
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 366ms/step
23
1/